In [1]:

import os
import google.generativeai as genai
import pandas as pd
# Gán API key (nên lấy từ biến môi trường)
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))  # Đặt API key vào biến môi trường GEMINI_API_KEY
model = genai.GenerativeModel("gemini-2.5-flash")

In [2]:
from prompt.analyze_data import analyze_dataset_prompt
from prompt.trainning_prompt import coding_instruct_prompt
#prompt_text = analyze_dataset_prompt("sensor_data.csv")
#print(prompt_text)

In [4]:
import os
import json
with open("idea.json", "r", encoding='utf-8') as f:
   idea_data = json.load(f)

with open("analysis/results.json", "r", encoding='utf-8') as f:
    results_data = json.load(f)

prompt_instruct = coding_instruct_prompt(idea_data, results_data)
print(prompt_instruct)


**Instructions for Implementation LLM: Generate Machine Learning Pipeline Prompt**

Your task is to analyze the dataset information located at `{'task': 'Train a classifier to predict whether a sensor reading window indicates an anomaly', 'goal': 'Optimize validation accuracy', 'initial_constraints': 'Use scikit-learn, or lSTM deep learning; preprocess data using time-windowing to capture temporal patterns'}` (which may be in CSV, JSON, or text format describing structure, features, task type, or statistics) and the research idea: '{'dataset_info': {'size': '1440 rows x 3 columns', 'type': 'multivariate time-series', 'data_types': ['float64', 'float64', 'float64']}, 'missing_values': {'sensor_1': '0.97%', 'sensor_2': '0.97%', 'sensor_3': '0.97%', 'handling_method': 'linear interpolation', 'reason': 'Linear interpolation is chosen as it is generally superior for time-series data. It estimates missing values based on neighboring points, thus preserving the temporal continuity and trends

In [5]:
responses = model.generate_content(prompt_instruct)
prompt = responses.text


In [8]:
print(prompt)

**Prompt for LLM to Generate Machine Learning Pipeline Script (`run_pipeline.py`)**

**Objective:** Generate a robust Python script (`run_pipeline.py`) to train a supervised machine learning classifier for anomaly detection in multivariate time-series sensor data. The script should cover data generation (for demonstration), preprocessing (including missing value imputation, time-windowing feature engineering, and scaling), model training, evaluation, and structured result saving.

**Dataset Information:**
*   **Source:** For demonstration purposes, the script should *simulate* a multivariate time-series dataset.
*   **Columns:** Three sensor readings: `sensor_1` (float), `sensor_2` (float), `sensor_3` (float).
*   **Size:** The simulated dataset should consist of 1440 rows, representing 1-minute intervals over a 24-hour period.
*   **Task:** Binary classification. The model needs to predict whether a given time window indicates an `is_anomaly` (binary: 0 for normal, 1 for anomaly).
*  

In [6]:
responsess = model.generate_content(prompt)
prompts = responsess.text


In [7]:
print(prompts)

```python
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import json

# --- Configuration ---
N_ROWS = 1440  # 24 hours * 60 minutes
MISSING_VALUE_RATE = 0.01
ROLLING_WINDOW_SIZE = 10  # 10 minutes
TEST_SPLIT_RATIO = 0.10
VAL_SPLIT_RATIO = 0.10
RANDOM_STATE = 42

# Define output directory and file
OUTPUT_DIR = 'result'
RESULTS_FILE = os.path.join(OUTPUT_DIR, 'results.json')

# --- Data Generation ---
print("1. Generating synthetic multivariate time-series data...")
# Generate time index for a 24-hour period (1-minute intervals)
time_index = pd.date_range(start='2023-01-01', periods=N_ROWS, freq='min')

# Simulate sensor data with some patterns
np.random.seed(RANDOM_STATE)
df = pd.DataFrame(index=time_index)
df['sensor_1'] = 10 * np.sin(np.linspace(0, 4 * np.pi, N_ROWS)) + np.random.normal(0, 0.5, N_ROW

In [9]:
import re
match = re.search(r"```(?:python)?\s*(.*?)```", prompts, re.DOTALL)
code = match.group(1).strip() if match else prompts.strip()

# 5. Ghi code ra file
with open("run_pipeline.py", "w", encoding="utf-8") as f:
    f.write(code)

print("✅ Code written to run_pipeline.py")

✅ Code written to run_pipeline.py
